In [1]:
import os

from tfx.components import (
    CsvExampleGen,
    ExampleValidator,
    SchemaGen,
    StatisticsGen,
    Transform,
)
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
from tfx.proto import example_gen_pb2

### Set Variable


In [2]:
PIPELINE_NAME = "sarcasm-pipeline"
SCHEMA_PIPELINE_NAME = "sarcasm-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join("../pipelines", PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join("../metadata", PIPELINE_NAME, "metadata.db")

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join("../serving_model", PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "../data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Data Ingestion


In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ]
    )
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

d:\Users\Jordan\VS-CODE\dicoding\ML\ml-ops\.venv\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Data Validation


In [9]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])

In [11]:
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [15]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

In [16]:
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [17]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'headline',BYTES,required,,-
'is_sarcastic',INT,required,,-


In [18]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"], schema=schema_gen.outputs["schema"]
)

In [19]:
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [22]:
interactive_context.show(example_validator.outputs["anomalies"])

### Data Preprocessing


In [25]:
TRANSFORM_MODULE_FILE = "../module/sarcasm_transform.py"

In [27]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=TRANSFORM_MODULE_FILE,
)

In [28]:
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: ../pipelines\sarcasm-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\55b5026c96c14f4c9bb2cd072c64bc92\assets


INFO:tensorflow:Assets written to: ../pipelines\sarcasm-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\55b5026c96c14f4c9bb2cd072c64bc92\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))